In [1]:
import re
import numpy as np
import keras
from keras.preprocessing.text import Tokenizer
from keras.layers import SimpleRNN, Dense, Input
from keras.models import Sequential

In [2]:
with open("./train_data.txt", "r", encoding="utf-8") as f:
    texts = f.read()
    texts = texts.replace("\ufeff", "")  # убираем первый невидимый символ
texts

'Вы — лучший ответ на проблемы, которые возникли в понедельник.\nДумайте позитивно и верьте в свою способность достигать отличных результатов.\nЕсли вы смогли в понедельник подняться с постели, значит вы супер герой.\nБудущее зависит от того, что вы делаете сегодня.\n"Неважно, как медленно вы идете, главное — не останавливаться."\n"Великие дела начинаются с малых шагов."\n"Не бойтесь провалов, они — часть пути к успеху."\n"Сделай сегодня то, что другие не хотят, и завтра будешь жить так, как другие не могут."\n"Ваше настроение — ваше оружие, не давайте ему сломаться."\n'

In [3]:
maxWordsCount = 2000
tokenizer = Tokenizer(
    num_words=maxWordsCount,
    filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»',
    lower=True,
    split=" ",
    char_level=False,
)
tokenizer.fit_on_texts([texts])
# tokenizer.index_word

In [4]:
dist = list(tokenizer.word_counts.items())
print(dist[:10])

[('вы', 5), ('лучший', 1), ('ответ', 1), ('на', 1), ('проблемы', 1), ('которые', 1), ('возникли', 1), ('в', 3), ('понедельник', 2), ('думайте', 1)]


In [5]:
data = tokenizer.texts_to_sequences([texts])
# data

In [6]:
res = keras.utils.to_categorical(data[0], num_classes=maxWordsCount)
print(res.shape)

(85, 2000)


In [7]:
inp_words = 3
n = res.shape[0] - inp_words

In [8]:
X = np.array([res[i : i + inp_words, :] for i in range(n)])
Y = res[inp_words:]

In [9]:
model = Sequential()
model.add(Input((inp_words, maxWordsCount)))
model.add(SimpleRNN(128, activation="tanh", return_sequences=True))  # Добавляем еще один слой SimpleRNN
model.add(SimpleRNN(128, activation="tanh"))
model.add(Dense(2000, activation="softmax"))  # Увеличиваем количество нейронов в слое Dense
model.summary()

model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer="adam")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 3, 128)            272512    
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 128)               32896     
                                                                 
 dense (Dense)               (None, 2000)              258000    
                                                                 
Total params: 563,408
Trainable params: 563,408
Non-trainable params: 0
_________________________________________________________________


In [10]:
history = model.fit(X, Y, batch_size=32, epochs=100)

Epoch 1/100
3/3 [==============================] - 3s 13ms/step - loss: 7.6023 - accuracy: 0.0000e+00
Epoch 2/100
3/3 [==============================] - 0s 11ms/step - loss: 7.5416 - accuracy: 0.1463
Epoch 3/100
3/3 [==============================] - 0s 12ms/step - loss: 7.4753 - accuracy: 0.5366
Epoch 4/100
3/3 [==============================] - 0s 12ms/step - loss: 7.3808 - accuracy: 0.4878
Epoch 5/100
3/3 [==============================] - 0s 11ms/step - loss: 7.2245 - accuracy: 0.2073
Epoch 6/100
3/3 [==============================] - 0s 11ms/step - loss: 6.9401 - accuracy: 0.0854
Epoch 7/100
3/3 [==============================] - 0s 10ms/step - loss: 6.4490 - accuracy: 0.0610
Epoch 8/100
3/3 [==============================] - 0s 10ms/step - loss: 5.7272 - accuracy: 0.0610
Epoch 9/100
3/3 [==============================] - 0s 11ms/step - loss: 5.0035 - accuracy: 0.0610
Epoch 10/100
3/3 [==============================] - 0s 11ms/step - loss: 4.5550 - accuracy: 0.0610
Epoch 11/100
3/

In [15]:
def buildPhrase(texts, str_len=21):
    res = texts
    data = tokenizer.texts_to_sequences([texts])[0]
    for i in range(str_len):
        x = keras.utils.to_categorical(
            data[i : i + inp_words], num_classes=maxWordsCount
        )  # преобразуем в One-Hot-encoding
        inp = x.reshape(1, inp_words, maxWordsCount)

        pred = model.predict(inp)  # предсказываем OHE четвертого символа
        indx = pred.argmax(axis=1)[0]
        data.append(indx)

        res += f" {tokenizer.index_word[indx]}"

    return res

In [16]:
res = buildPhrase("не бойтесь провалов")
print(res)

1/1 [==============================] - 0s 26ms/step
не бойтесь провалов они часть пути к успеху сделай сегодня то что другие не хотят и завтра будешь жить так как другие не могут
